In [1]:
#!pip install jieba
import os
import jieba
import pandas as pd

In [2]:
# 將印出來的rows只印出15行，columns只印出10行
pd.set_option('display.max_rows', 15)
pd.set_option('display.max_columns', 10)

In [3]:
# scikit-learn 會有些 deprecation warning, 為了顯示漂亮, 刻意地忽略掉
import warnings
warnings.filterwarnings('ignore')

In [4]:
base_dir = "./NLP_Naive_Bayes_newsfile/chinese_news_trans"
test_dir = "./NLP_Naive_Bayes_newsfile/chinese_news_test"

In [5]:
# 因為要處理資料夾很多次,所以定義成函式
def process_dirs(base_dir):
    df = pd.DataFrame(columns = ["類別", "內容"])
    # os.walk 會走到檔案才停下來
    for dir_path, dir_names, file_names in os.walk(base_dir):
        for single_file in file_names:
            if not single_file.startswith("."):
                f = open(os.path.join(dir_path, single_file), "r", encoding = "utf-8")
                content = f.read()
                # 讀完檔案以後做出第一步處理, 先把換行都去掉
                content = content.replace("\r", "").replace("\n", "")
                split_word = jieba.cut(content)
                # 分詞
                content = " ".join(split_word)
                s = pd.Series([dir_path.split("/")[-1], content], index = ["類別", "內容"])
                df = df.append(s, ignore_index = True)
    df['類別'] = df['類別'].astype('category')
    return df

In [6]:
df = process_dirs(base_dir)

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.182 seconds.
Prefix dict has been built succesfully.


In [7]:
df

,類別,內容
0,體育,國家 體委 和 全國體 總電 賀三國 和平 登山...
1,體育,（ 專發 《 天津 日報 》 ） 三國 和平 登...
2,體育,王志良 透露 亞運會 集資部 活動 經費 支出 ...
3,體育,通訊 ： 聶衛平 從 嚴治軍新華社 杭州 ５ 月...
4,體育,西藏自治 區 政府 宴請 和平 登山 隊新華 社...
5,體育,瑞典 男隊 獲首屆 世界 盃 乒乓球 團體賽 冠...
6,體育,女子 冠 軍 雷吉娜 · 傑 奎斯 在 隨後進行 的 是 男子 花樣 滑 決賽 把 ...
...,...,...
2630,經濟,廣東省 一季度 財政 收入 繼續 增長 新華社 ...
2631,經濟,我國 第一 個 證券 信息 網在滬 成立 新華社...


由於 scikit-learn 不接受字串，所以我們一定要把類別轉換成整數，可以使用 cat.categories 得到所有類別，再使用 cat.codes 轉換成整數。

In [8]:
# 把類別替換成 code ,並且記錄起來
# 走過 categories 同時順便把字典創造起來
saved_map = { 
    cat:df['類別'].cat.categories.get_loc(cat)
        for cat in df['類別'].cat.categories 
    }

In [9]:
saved_map

{'交通': 0,
 '政治': 1,
 '教育': 2,
 '環境': 3,
 '經濟': 4,
 '藝術': 5,
 '計算機': 6,
 '軍事': 7,
 '醫藥': 8,
 '體育': 9}

In [10]:
df['類別'] = df['類別'].cat.codes

In [11]:
df

,類別,內容
0,9,國家 體委 和 全國體 總電 賀三國 和平 登山...
1,9,（ 專發 《 天津 日報 》 ） 三國 和平 登...
2,9,王志良 透露 亞運會 集資部 活動 經費 支出 ...
3,9,通訊 ： 聶衛平 從 嚴治軍新華社 杭州 ５ 月...
4,9,西藏自治 區 政府 宴請 和平 登山 隊新華 社...
5,9,瑞典 男隊 獲首屆 世界 盃 乒乓球 團體賽 冠...
6,9,女子 冠 軍 雷吉娜 · 傑 奎斯 在 隨後進行 的 是 男子 花樣 滑 決賽 把 ...
...,...,...
2630,4,廣東省 一季度 財政 收入 繼續 增長 新華社 ...
2631,4,我國 第一 個 證券 信息 網在滬 成立 新華社...


In [12]:
test_df = process_dirs(test_dir)

In [13]:
test_df

,類別,內容
0,體育,各國 記者 眼中 的 羽毛球 世錦賽 - - - - - - - - - - - -...
1,體育,最 優秀 選手 無緣 亞運會 健美 賽 健美 在 亞洲 運動會 ...
2,體育,帆 板 運 動 簡 介 （ 二 ） 我國 在 79 年 由 國家...
3,體育,男子 健美 初登亞 運會 中國 猛男 直指 前 三 在 即將...
4,體育,不解 之 謎 第 2 屆 奧運會 在 法國 巴黎 舉行 。 這次 奧運會 的 馬 拉...
5,體育,亞運會 游泳 賽中國 臺 北隊 改寫 歷史 新華社 曼谷 １ ２ 月...
6,體育,中國 青島 － 韓國 大邱 健美 表演 賽 和 對 抗賽 將舉 辦 ...
...,...,...
94,經濟,日 月光 華 - - Economics 精華區 文章 閱讀 - - - - - -...
95,經濟,日 月光 華 - - Economics 精華區 文章 閱讀 - - - - - -...


這邊必須使用剛剛存起來的字典來替換，因為如果直接使用 code 可能會發生沒對照到的事故。

In [14]:
test_df['類別'] = test_df['類別'].replace(saved_map)

In [15]:
test_df

,類別,內容
0,9,各國 記者 眼中 的 羽毛球 世錦賽 - - - - - - - - - - - -...
1,9,最 優秀 選手 無緣 亞運會 健美 賽 健美 在 亞洲 運動會 ...
2,9,帆 板 運 動 簡 介 （ 二 ） 我國 在 79 年 由 國家...
3,9,男子 健美 初登亞 運會 中國 猛男 直指 前 三 在 即將...
4,9,不解 之 謎 第 2 屆 奧運會 在 法國 巴黎 舉行 。 這次 奧運會 的 馬 拉...
5,9,亞運會 游泳 賽中國 臺 北隊 改寫 歷史 新華社 曼谷 １ ２ 月...
6,9,中國 青島 － 韓國 大邱 健美 表演 賽 和 對 抗賽 將舉 辦 ...
...,...,...
94,4,日 月光 華 - - Economics 精華區 文章 閱讀 - - - - - -...
95,4,日 月光 華 - - Economics 精華區 文章 閱讀 - - - - - -...


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer()
# 注意一定要使用 fit_transform ,才會幫你轉換成詞向量
bag = vec.fit_transform(df['內容'])
print("總共維度:", len(vec.get_feature_names()))

總共維度: 96042


In [17]:
# 可以看到 transform 後的 matrix
#for entry in bag:
#    print(entry)

scikit-learn 單純貝氏
1.BernoulliNB: 對於特徵是 True 和 False 二分法優化。
2.GaussianNB: 對於特徵是高斯分布的連續數字優化。
3.MultinomialNB: 對於特徵是整數,而且是數幾次的分布優化,不過使用就算使用 tf-idf 分數一樣可以達到很好效果。

In [18]:
# 只要是文字, 我們通常就會選擇 MultinomialNB
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha = 0.001).fit(bag, df['類別'])

In [19]:
# 由於我們用剛剛的 vec 訓練他, 所以維度會保持跟剛剛一樣
test_bag = vec.transform(test_df['內容'])
print("維度:", len(vec.get_feature_names()))

維度: 96042


In [20]:
from sklearn.metrics import accuracy_score

predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("Naive-Bayes 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
正確標籤: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
Naive-Bayes 正確率:  100.0 %


In [21]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(n_neighbors=8)
clf = clf.fit(bag, df['類別'])
predict = clf.predict(test_bag)
print("預測:", list(predict))
print("正確標籤:", list(test_df['類別']))
print("kNN 正確率: ", accuracy_score(test_df['類別'], predict) * 100, "%")

預測: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
正確標籤: [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]
kNN 正確率:  100.0 %
